# **Download and Explore Satellite Imagery Data for Floods**

In [3]:
import rasterio as rs
from PIL import Image
import random, os, numpy as np
from time import time
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
import warnings 
warnings.filterwarnings('ignore')

## Sen1Floods11

This data's citation is:
> Bonafilia, D., Tellman, B., Anderson, T., Issenberg, E. 2020. Sen1Floods11: a georeferenced dataset to train and test deep learning flood algorithms for Sentinel-1. The IEEE/CVF Conference on Computer Vision and Pattern Recognition (CVPR) Workshops, 2020, pp. 210-211.

[The paper is available via open access](http://openaccess.thecvf.com/content_CVPRW_2020/html/w11/Bonafilia_Sen1Floods11_A_Georeferenced_Dataset_to_Train_and_Test_Deep_Learning_CVPRW_2020_paper.html)

The GCS bucket is split into subfolders containing data, checkpoints, training/testing splits and a [STAC](https://stacspec.org) compliant catalogue.

For additional details on how the data is structured and what is represented in the meta data, head over to the [github repo](https://github.com/cloudtostreet/Sen1Floods11/tree/master)

In [13]:
path_data = Path('/gis-exploration/data/downloads')

In [29]:
!ls -l /home/ba_ch/nbs/gis-exploration/data/downloads/v1.1/

total 10252
-rw-r--r-- 1 ba_ch ba_ch    14092 Oct 15  2020 Sen1Floods11_Metadata.geojson
drwxr-xr-x 6 ba_ch ba_ch     4096 Mar 10 16:03 catalog
-rw-r--r-- 1 ba_ch ba_ch 10467733 Oct 15  2020 catalog.zip
drwxr-xr-x 2 ba_ch ba_ch     4096 Mar 10 16:13 checkpoints
drwxr-xr-x 3 ba_ch ba_ch     4096 Mar 10 16:04 data
